# Multiple linear regression and adjusted R-squared

## Import the relevant libraries

In [244]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn
seaborn.set()

# the actual regression (machine learning) module
from sklearn.linear_model import LinearRegression

## Load the data from a .csv in the same folder

In [245]:
data = pd.read_csv('/Users/ranjanadobal/Documents/data/MultipleLR/1.02. Multiple linear regression.csv')

### Let's check what's inside this data frame

In [246]:
data.head()

,SAT,GPA,"Rand 1,2,3"
0,1714,2.40,1
1,1664,2.52,3
2,1760,2.54,3
3,1685,2.74,3
4,1693,2.83,2


#There are two input features, SAT score and Rand 1,2,3 and the dependent variable is GPA.Rand1,2,3 is a variable which randomly assigns 1,2,3 to each sample. 


### Print the descriptive statistics for each column in the data frame.

In [247]:
data.describe()

,SAT,GPA,"Rand 1,2,3"
count,84.000000,84.000000,84.000000
mean,1845.273810,3.330238,2.059524
std,104.530661,0.271617,0.855192
min,1634.000000,2.400000,1.000000
25%,1772.000000,3.190000,1.000000
50%,1846.000000,3.380000,2.000000
75%,1934.000000,3.502500,3.000000
max,2050.000000,3.810000,3.000000


## Multiple Regression

### Declare the dependent and independent variables

#There are two independent variables: 'SAT' and 'Rand 1,2,3' and a single depended variable: 'GPA'

In [248]:
x = data[['SAT','Rand 1,2,3']]

y = data['GPA']

### Regression itself

#We start by creating a linear regression object and then we fit the regression

In [249]:
reg = LinearRegression()

reg.fit(x,y)

LinearRegression()

#Getting the coefficients of the regression. The output is an array. 

In [250]:
reg.coef_

array([ 0.00165354, -0.00826982])

#The coefficients are ordered in the way they were fed in the model. 0.0017 is the coefficient for SAT and -0.0083 is the coefficient for Rand1,2,3.  

#Getting the intercept of the regression. The result is a float as we usually expect a single value

In [251]:
reg.intercept_

0.29603261264909486

#The intercept of the model is 0.296. 

### Calculating the R-squared. 

#Get the R-squared of the regression. R-squared is the most common measures of goodness of fit. The same score() method can be used for both simple linear regression and multiple linear regression. Score method takes two arguments - inputs and target. 

In [252]:
reg.score(x,y)

0.40668119528142843

#The R-squared value is exactly the same as the statsmodels summary. Ajd R-squared is a more appropriate measure for multiple linear regression. There is no method in sklearn to calculate Adj R-squared directly. 

### Formula for Adjusted R^2

$R^2_{adj.} = 1 - (1-R^2)*\frac{n-1}{n-p-1}$

#In this formula, N is the number of observations i.e. 84 and p is the number of predictors i.e. 2. 

#Get the shape of x, to facilitate the creation of the Adjusted R^2 metric

In [253]:
x.shape

(84, 2)

#If we want to find the Adjusted R-squared we can do so by knowing the r2, the # observations, the # features.  
Number of observations is the shape along axis 0. 
Number of features (predictors, p) is the shape along axis 1. 
We find the Adjusted R-squared using the formula

In [254]:
r2 = reg.score(x,y)

n = x.shape[0]

p = x.shape[1]


adjusted_r2 = 1-(1-r2)*(n-1)/(n-p-1)
adjusted_r2

0.39203134825134023

#The Adj R-squared is  0.392 which is lesser than R-squared 0.407 and this means one of the predictors has very little explanatory power. 

## Feature selection 

#Feature Selection is a process created to detect the variables which are not needed in a model. 
#Import the feature selection module from sklearn
#This module allows us to select the most appopriate features for our regression
#There exist many different approaches to feature selection, however, this is one of the simplest.

This process helps improve speed and prevents other issues due to too many features. This process uses the p-values of variables and discards the variables with p-value above 0.05. 

In [255]:
from sklearn.feature_selection import f_regression

#We will look into: f_regression
#f_regression finds the F-statistics for the *simple* regressions created with each of the independent variables
#In our case, this would mean running a simple linear regression on GPA where SAT is the independent variable
#and a simple linear regression on GPA where Rand 1,2,3 is the indepdent variable
#The limitation of this approach is that it does not take into account the mutual effect of the two features.
Call the method f_regression with arguments the features contained in X and the target contained in Y.


In [256]:
f_regression(x,y)

(array([56.04804786,  0.17558437]), array([7.19951844e-11, 6.76291372e-01]))

#There are two output arrays
#The first one contains the F-statistics for each of the regressions
#The second one contains the p-values of these F-statistics

#Since we are more interested in the latter (p-values), we can just take the second and store it in a new variable 'p-values'. 

In [257]:
p_values = f_regression(x,y)[1]
p_values

array([7.19951844e-11, 6.76291372e-01])

#e-11 means multiplied by 10 to the power -11. 

#To be able to quickly evaluate them, we can round the result to 3 digits after the dot.  

In [258]:
p_values.round(3)

array([0.   , 0.676])

#The p-value of the first column SAT is 0.000 and the p-value of the second column Rand1,2,3 is 0.676. Rand1,2,3 variable is not significant in the model. These p-values do not reflect the interconnection of these variables in the model. 

## Creating a summary table 

#Let's create a new data frame with the names of the features. 

In [259]:
reg_summary = pd.DataFrame(data = x.columns.values, columns=['Features'])
reg_summary

,Features
0,SAT
1,"Rand 1,2,3"


#Then we create and fill a second column, called 'Coefficients' with the coefficients of the regression. # Finally, we add the p-values we just calculated

In [260]:
reg_summary ['Coefficients'] = reg.coef_
reg_summary ['p-values'] = p_values.round(3)

#Now we've got a pretty clean summary, which can help us make an informed decision about the inclusion of the variables 

In [261]:
reg_summary

,Features,Coefficients,p-values
0,SAT,0.001654,0.000
1,"Rand 1,2,3",-0.008270,0.676


The p-values help to determine if a variable is redundant, but they provide no information about how useful a variable is.

## Feature Scaling or standardization

#feature scaling is the process of transforming the data we are working with into a standard scale. This is achieved by subtracting the mean and dividing by the standard deviation. This way,regardless of the data set we will always obtain a distribution with a mean of zero and a standard deviation of one. 

#Import the preprocessing module StandardScaler from sklearn and is one of the easiest and 'cleanest' ways to preprocess your data

In [262]:
from sklearn.preprocessing import StandardScaler

#Create a StandardScaler object which is empty. 

In [263]:
scaler = StandardScaler()

In [264]:
scaler

StandardScaler()

#Fit the input data (x). Essentially, we are calculating the mean and standard deviation feature-wise (the mean of 'SAT' and the standard deviation of 'SAT', as well as the mean of 'Rand 1,2,3' and the standard deviation of 'Rand 1,2,3'). This line will calculate the mean and standard deviation of each feature. This information will be stored
in the scaler object. This scaler object will now contain information about the mean and standard deviation of the data. Whenever we get new data,we know that the standardization information is contained in the scaler object.


In [265]:
scaler.fit(x)

StandardScaler()

#The inputs are still unscaled. The actual scaling of the data is done through the method 'transform()'
#Let's store it in a new variable x_scaled. transform() method transforms the unscaled inputs using the information contained in scaler and that means we subtract the mean and divide by the standard deviation for each feature. Whenever we get new data we will just apply scaler.transform new data to reach the same transformation as we just did.

In [266]:
x_scaled = scaler.transform(x)

#The result is an ndarray and all the input data has been standardized.


In [267]:
x_scaled

array([[-1.26338288, -1.24637147],
       [-1.74458431,  1.10632974],
       [-0.82067757,  1.10632974],
       [-1.54247971,  1.10632974],
       [-1.46548748, -0.07002087],
       [-1.68684014, -1.24637147],
       [-0.78218146, -0.07002087],
       [-0.78218146, -1.24637147],
       [-0.51270866, -0.07002087],
       [ 0.04548499,  1.10632974],
       [-1.06127829,  1.10632974],
       [-0.67631715, -0.07002087],
       [-1.06127829, -1.24637147],
       [-1.28263094,  1.10632974],
       [-0.6955652 , -0.07002087],
       [ 0.25721362, -0.07002087],
       [-0.86879772,  1.10632974],
       [-1.64834403, -0.07002087],
       [-0.03150724,  1.10632974],
       [-0.57045283,  1.10632974],
       [-0.81105355,  1.10632974],
       [-1.18639066,  1.10632974],
       [-1.75420834,  1.10632974],
       [-1.52323165, -1.24637147],
       [ 1.23886453, -1.24637147],
       [-0.18549169, -1.24637147],
       [-0.5608288 , -1.24637147],
       [-0.23361183,  1.10632974],
       [ 1.68156984,

## Regression with scaled features

In the earlier regression, the model was GPA = 0.296 + 0.0017SAT -0.0083Rand1,2,3 and by looking at the coefficients alone, it seems like Rand1,2,3 has a bigger coefficient and hence a greater impact.The SAT expression
is contributing a lot more to the final expression overall, as 1800(SAT score) times 0.0017 equals 3.06, while two (Rand1,2,3) times minus 0.0083 equals 0.0166.
This issue is overcome through feature scaling. Having all inputs with the same magnitude allows us to compare their impact.
#Creating a regression works in the exact same way. # We just need to specify that our inputs are the 'scaled inputs' and specify the targets. 

In [268]:
reg = LinearRegression()


reg.fit(x_scaled,y)

LinearRegression()

#To look at the actual difference in regression, we can see the coefficients and intercept. 

#Let's see the coefficients

In [269]:
reg.coef_

array([ 0.17181389, -0.00703007])

#And the intercept

In [270]:
reg.intercept_

3.330238095238095

## Creating a summary table

#As usual we can try to arrange the information in a summary table
#Let's create a new data frame with the names of the features. 
#Then we create and fill a second column, called 'Weights' with the coefficients of the regression
#Since the standardized coefficients are called 'weights' in ML, this is a much better word choice for our case
#Note that even non-standardized coeff. are called 'weights' 
#but more often than not, when doing ML we perform some sort of scaling. The name 'weights' comes from the fact that the bigger the weight the bigger the impact of the feature on the regression. In a machine learning context, the intercept is called bias. The idea is that the intercept is nothing but a number which adjusts our regression with some constant. But if we need to adjust our regression with some constant then the regression is biased by that number.


In [271]:
reg_summary = pd.DataFrame([['Bias'],['SAT'],['Rand 1,2,3']], columns=['Features'])

reg_summary['Weights'] = reg.intercept_, reg.coef_[0], reg.coef_[1]

#Now we have a pretty clean summary, which can help us make an informed decision about the importance of each feature. The closer a weight is to zero, the smaller it's impact. The bigger the weight,the bigger it's impact.
 

In [272]:
reg_summary

,Features,Weights
0,Bias,3.330238
1,SAT,0.171814
2,"Rand 1,2,3",-0.007030


#This brings us to feature selection through standardization, we can clearly see that Rand 1,2, 3barely contributes to our outputs, if at all. It will make little difference if we remove it from the model or leave it there with a weight of almost zero.

When we perform feature scaling, we don't really care if a useless variable is there or not. This is also one of the main reasons why Sklearn does not natively support P values. Since most ML practitioners perform some kindof feature scaling before fitting a model, we don't really need to identify the worst performing features. They are automatically penalized by having a very small weight.

In general, it is better to leave out the worst performing features as they interact with the useful ones and may bias the weights even if only slightly so.

## Making predictions with the standardized coefficients (weights)

#For simplicity, let's create a new dataframe with 2 *new* observations. One is a student who got 1700
on the SAT and was assigned the number two randomly while the other had 1800 on the SAT and was assigned one randomly. The new dataframe should be arranged in the exact same way as our input data, X and must be standardized
in the same way, with the same mean and standard deviation as we stored in the scaler object. 



In [273]:
new_data = pd.DataFrame(data=[[1700,2],[1800,1]],columns=['SAT','Rand 1,2,3'])
new_data

,SAT,"Rand 1,2,3"
0,1700,2
1,1800,1


#This is to ensure that the output is GPA because the regression model was trained on standardized inputs. We see that the predict method on the unscaled inputs gives an unexpected GPA output below. 

In [274]:
reg.predict(new_data)

/Applications/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


array([295.39979563, 312.58821497])

##Our model is expecting SCALED features (features of different magnitude), so we must transform the 'new data' in the same way as we transformed the inputs we train the model on. This information is contained in the 'scaler' object. We simply transform the 'new data' using the relevant method. Let's scale the new_data. 

In [275]:
new_data_scaled = scaler.transform(new_data)

new_data_scaled

array([[-1.39811928, -0.07002087],
       [-0.43571643, -1.24637147]])

#The result is an ND array which contains the standardized data. Now that looks exactly like the inputs we train the model with.

#We can make a prediction for a whole dataframe (not a single value) by calling the predict method on the regression
and then specifying the new inputs as an argument.  Using the well-known method, predict we can find what the predictions are when we feed the new data scaled array. Here's the result, it is of the magnitude we anticipated.
Our first student is predicted to have a GPAof 3.09 while the second 3.26.

#Finally we make a prediction using the scaled new data and it gives an expected GPA output. 

In [276]:
reg.predict(new_data_scaled)

array([3.09051403, 3.26413803])

#What if we removed the 'Random 1,2,3' variable?

#Theory suggests that features with very small weights could be removed and the results should be identical
#Moreover, we proved in 2-3 different ways that 'Rand 1,2,3' is an irrelevant feature
#Let's create a simple linear regression (simple, because there is a single feature) without 'Rand 1,2,3'

In [277]:
reg_simple = LinearRegression()

#Once more, we must reshape the inputs into a matrix or a 2D array, otherwise we will get a compatibility error. #In order to feed x to sklearn, it should be a 2D array (a matrix).Therefore, we must reshape it. Note that we saw this in Simple Linear Regression that this is not needed when we've got more than 1 feature (as the inputs will be a 2D array by default). x_matrix = x.values.reshape(84,1). reshape(-1,1) is a more generalized approach. 
#Note that instead of standardizing again, I'll simply take only the first column of x.

In [278]:
x_simple_matrix = x_scaled[:,0].reshape(-1,1)

#Finally, we fit the regression

In [279]:
reg_simple.fit(x_simple_matrix,y)

LinearRegression()

#In a similar manner to the cell before, we can predict only the first column of the scaled 'new data'
#Note that we also reshape it to be exactly the same as x

In [280]:
reg_simple.predict(new_data_scaled[:,0].reshape(-1,1))

array([3.08970998, 3.25527879])

#The predicted GPA is slightly different than what we received from or multiple linear regression earlier, but actually if we round up to two digits after the dot we get the exact same results 3.09 and 3.26. This finding shows us  that P values are not needed in sklearn. When we apply feature scaling it often does not affect the final result if We keep or leave out in significant features. Their weights will be so close to zero that they will barely influence the predictions.